In [1]:
using JuMP
using DecisionProgramming

ArgumentError: ArgumentError: Package DecisionProgramming not found in current path.
- Run `import Pkg; Pkg.add("DecisionProgramming")` to install the DecisionProgramming package.

In [2]:
N = 4
diagram = InfluenceDiagram()

add_node!(diagram, ChanceNode("h1", [], ["ill", "healthy"])) # creates initial health state with no information state

for i in 1:N-1
    add_node!(diagram, ChanceNode("t$i", ["h$i"], ["positive", "negative"])) # test result
    add_node!(diagram, DecisionNode("d$i", ["t$i"], ["treat", "not treat"])) # treatment decision
    add_node!(diagram, ValueNode("u$i", ["d$i"])) # test cost
    add_node!(diagram, ChanceNode("h$(i+1)", ["h$i", "d$i"], ["ill", "healthy"])) # health state in next period
end 

add_node!(diagram, ValueNode("u$N", ["h$N"])) # value node with market value

generate_arcs!(diagram)

diagram

UndefVarError: UndefVarError: `InfluenceDiagram` not defined

In [3]:
add_probabilities!(diagram, "h1", [0.1, 0.9]) # initial health state probability

# probablity matrix specifying health conditions given previous health and treatment decision
X_h = ProbabilityMatrix(diagram, "h2")
X_h["healthy", "not treat", :] = [0.2, 0.8]
X_h["healthy", "treat", :] = [0.1, 0.9]
X_h["ill", "not treat", :] = [0.9, 0.1]
X_h["ill", "treat", :] = [0.5, 0.5]

X_h

UndefVarError: UndefVarError: `diagram` not defined

In [4]:
X_t = ProbabilityMatrix(diagram, "t1")
X_t["ill", "positive"] = 0.8
X_t["ill", "negative"] = 0.2
X_t["healthy", "negative"] = 0.9
X_t["healthy", "positive"] = 0.1

X_t

UndefVarError: UndefVarError: `ProbabilityMatrix` not defined

In [5]:
for i in 1:N-1
    add_probabilities!(diagram, "t$i", X_t)
    add_probabilities!(diagram, "h$(i+1)", X_h)
end

UndefVarError: UndefVarError: `diagram` not defined

In [6]:
for i in 1:N-1
    add_utilities!(diagram, "u$i", [-100.0, 0.0])
end

add_utilities!(diagram, "u$N", [300.0, 1000.0]);

UndefVarError: UndefVarError: `diagram` not defined

In [7]:
generate_diagram!(diagram, positive_path_utility=true);

UndefVarError: UndefVarError: `generate_diagram!` not defined

In [8]:
using Cbc
using Gurobi
using HiGHS

SOLVER = Gurobi

model = Model(SOLVER.Optimizer)

z = DecisionVariables(model, diagram)
x_s = PathCompatibilityVariables(model, diagram, z, probability_cut = false)

expected_profit = expected_value(model, diagram, x_s)
@objective(model, Max, expected_profit)

model

ArgumentError: ArgumentError: Package Cbc not found in current path.
- Run `import Pkg; Pkg.add("Cbc")` to install the Cbc package.

In [9]:
optimize!(model)

UndefVarError: UndefVarError: `model` not defined

In [10]:
# Retriving solution elements
Z = DecisionStrategy(z)
S_probabilities = StateProbabilities(diagram, Z)
U_distribution = UtilityDistribution(diagram, Z);

print_decision_strategy(diagram, Z, S_probabilities)

UndefVarError: UndefVarError: `DecisionStrategy` not defined

In [11]:
print_utility_distribution(U_distribution)
print_statistics(U_distribution)

UndefVarError: UndefVarError: `print_utility_distribution` not defined

In [12]:
using Plots
p_cumulative = [sum(U_distribution.p[1:i]) for i = 1:length(U_distribution.p)]
scatter(U_distribution.u, p_cumulative,
    label = false,
    xaxis = ("Market value", 0:100:1000),
    yaxis = ("Probability", 0.0:0.1:1.0),
    title = "Revenue curve" 
    )


UndefVarError: UndefVarError: `U_distribution` not defined